Tutorial Followed:

https://snawarhussain.com/blog/computer%20vision/python/tutorial/photometric-stereo-lambertian-model/

Loading dataset and normalizing it with the intensity values

In [2]:
import random
import cv2
import glob
import re
import os
import numpy as np

In [3]:
def numerical_sort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def imread(file_name, intensities=None, flag=-1, scale=1.0):
    img_names = glob.glob(file_name)
    img_list = []
    if len(img_names) == 0:
        print("[No {} images]".format(file_name))
        exit()
    img_names = sorted(img_names, key=numerical_sort)
    for i, path in enumerate(img_names):
        img = cv2.resize(cv2.imread(path, flag), None, fx=scale, fy=scale)
        if  np.any(intensities):
            img = img / intensities[i]
        img_list.append(img)
    return img_list

In [ ]:
from numpy import loadtxt

dataPath = "data/"
object = []
for obj  in (os.listdir(dataPath)):
    object.append(obj)

L_direction = loadtxt(f"data/{object[0]}/info/light_directions.txt")
intensities = loadtxt(f"data/{object[0]}/info/light_intensities.txt")
path_img = "data/bearPNG"

imgs = imread(path_img + f"/{object[0]}/*", intensities=intensities)
mask = imread(f"data/{object[0]}/info/mask.png")[0]

# Applying the mask to the image.
for i, bear in enumerate(imgs):
    imgs[i] = cv2.bitwise_and(bear, bear, mask=mask)
    i += 1

imgs = np.array(imgs)

Photometric Stereo Function

In [ ]:
def PMS(imgs, L_list):
    """
    This is the main function for performing photometric stereo using three images.

        Parameters
        ----------
        imgs: np.ndarray
            ndarray of all the images of an object taken with different light sources
        L_list: list
            the list of the light sources directions S for each image. for each image the S is a vector matrix with 3  positional elements (x,y,z)
        return:
            returns the computed albedos and the surface normals of the object
    """

    L = np.array(L_list) # note that light vector is passed as a list.
    Lt = L.T # taking transpose.
    h, w = imgs[0].shape[:2] # note that images are passed as an ndarray.

    img_normal = np.zeros((h, w, 3)) 
    img_albedo = np.zeros((h, w, 3))
    I = np.zeros((len(L_list), 3)) # Image intensity.

    for x in range(w): # iterate through all rows.
        for y in range(h): # iterate through all columns.
            for i in range(len(imgs)): # iterate through all images.
                I[i] = imgs[i][y][x] # getting intensities for the 3 images to construct the Intensity vecotr

            tmp1 = np.linalg.inv(Lt) 

            N = np.dot(tmp1, I).T # based on the equation N = inv(S)*I, where S is the source matrix (includes all vectors)
            rho = np.linalg.norm(N, axis=1) # calculate albedo, axis = 1 calculates for every row, i.e for each normal vector in the array N.
            # specifically done for row because in N matrix, each row represents a point in 3D space. So each row needs to be treated as an independent vector.
            # if axis is not set then norm computes euclidean norm for the whole matrix as if it was a single flattened vector, this will yield a single scalarvalue.
            # is axis = 0, then euclidean norm will be computed column wise.
            img_albedo[y][x] = rho

            # Create a weighted projection of the normal vector into a gray-scale value. Just for representation.
            N_gray = N[0]*0.0722 + N[1]*0.7152 + N[2]*0.2126  # luminosity formula (PS: x, y, z weighed to Gray), i.e borrowed from RGB color space but not a color space conversion here.
            Nnorm = np.linalg.norm(N)
            if Nnorm==0: # finding degenerate cases, invalid dark pixels with no light.
                continue 
            img_normal[y][x] = N_gray/Nnorm # normalize the Ngray derived above.

    img_normal_rgb = ((img_normal*0.5 + 0.5)*255).astype(np.uint8)  # (x y z) cooridinate to RGB coordinate converstion. Equation in the tutorial.
    img_normal_rgb = cv2.cvtColor(img_normal_rgb, cv2.COLOR_BGR2RGB)  # converting from BGR to RGB for correct visualization. cv2 uses BGR format.
    img_albedo = (img_albedo/np.max(img_albedo)*255).astype(np.uint8)
    return img_normal, img_albedo, img_normal_rgb

Sample 3 images randomly from data for constructing surface normals and albedo

In [ ]:
import matplotlib.pyplot as plt
import random
random.seed(5)
index = random.sample(range(0,96), 3)
img_normal, img_albedo, img_normal_rgb = PMS(imgs[list(index),:], L_direction[list(index),:])

Plotting

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1,3,1)

img = (imgs[1] - np.min(imgs[1])) / (np.max(imgs[1]) - np.min(imgs[1]))  #normalizing image between 0-1 for visualization
plt.imshow(img)
plt.title('Example image with one light source')
plt.xticks([])
plt.yticks([])
#plt.plot([1,2,3,4])
plt.subplot(1,3,2)
#plt.plot([1,2,3,4])
plt.imshow(img_albedo)
plt.title('Image Albedo')
plt.xticks([])
plt.yticks([])
plt.subplot(1,3,3)
#plt.plot([1,2,3,4])
plt.imshow(img_normal_rgb)
plt.title('Surface Normals')
plt.xticks([])
plt.yticks([])
plt.show()